In [31]:
import mlflow
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

from mlflow.tracking import MlflowClient
from mlflow.utils.rest_utils import http_request
import uuid

# subscription_id = '96aede12-2f73-41cb-b983-6d11a904839b'
# resource_group = 'promptflow'
# workspace_name = 'promptflow-canary'

subscription_id = '96aede12-2f73-41cb-b983-6d11a904839b'
resource_group = 'hod-rg'
workspace_name = 'hod-pflow'

ml_client = MLClient(credential=DefaultAzureCredential(),
                        subscription_id=subscription_id,
                        resource_group_name=resource_group,
                        workspace_name=workspace_name)

mlflow_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
print(mlflow_tracking_uri)
mlflow.set_tracking_uri(mlflow_tracking_uri)

azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourceGroups/hod-rg/providers/Microsoft.MachineLearningServices/workspaces/hod-pflow


In [26]:
run = mlflow.start_run(run_id="a9cc702b-fdb3-4f25-945c-c48a7800299f")
print(run.info.run_id)

a9cc702b-fdb3-4f25-945c-c48a7800299f


In [25]:
print(mlflow.active_run())

None


In [ ]:
mlflow.end_run()

In [ ]:
run = mlflow.active_run()
print(run.info.run_id)

In [34]:

from enum import Enum
class Status(Enum):
    Started = "Started"
    Completed = "Completed"
    Failed = "Failed"
    Cancelled = "Cancelled"
    NOTSTARTED = 'NotStarted'

    @staticmethod
    def is_terminated(status):
        return status in {Status.Completed, Status.Failed, Status.Cancelled}
    
    
print(Status.is_terminated(Status.Completed))

True


In [2]:
class AzureMLConfig:
    SUBSCRIPTION_ID = "SUBSCRIPTION_ID"
    RESOURCE_GROUP_NAME = "RESOURCE_GROUP_NAME"
    WORKSPACE_NAME = "WORKSPACE_NAME"
    WORKSPACE_REGION = "WORKSPACE_REGION"
    
    @staticmethod
    def test():
        print("test")

class hod(AzureMLConfig):
    a = 1

hod.test()

test


In [7]:


run_id = str(uuid.uuid4())
print(f"New run id: {run_id}")

# get auth from client
client = MlflowClient()
cred = client._tracking_client.store.get_host_creds()
cred.host = cred.host.replace("mlflow/v2.0", "mlflow/v1.0").replace("mlflow/v1.0", "history/v1.0")
response = http_request(
    host_creds=cred,
    endpoint="/experiments/{}/runs/{}".format("promptflow", run_id),
    method="PATCH",
    json={"runId": run_id},
)
if response.status_code == 200:
    print(f"Successfully created a run with run id '{run_id}'")
else:
    raise


New run id: d4a2d5c7-57f9-4dec-848e-19d3969eb22f
Successfully created a run with run id 'd4a2d5c7-57f9-4dec-848e-19d3969eb22f'


## Cancel an active run

In [5]:

current_run = mlflow.active_run()
run_id = current_run.info.run_id
experiment_id = current_run.info.experiment_id

# get auth from client
client = MlflowClient()
cred = client._tracking_client.store.get_host_creds()
cred.host = cred.host.replace("mlflow/v2.0", "mlflow/v1.0").replace("mlflow/v1.0", "history/v1.0")

end_event = {
    "timestamp": datetime.utcnow().isoformat(),
    "name": "Microsoft.MachineLearning.Run.Canceled",
    "data": {
        "endtime": datetime.utcnow().isoformat(),
    },
}
response = http_request(
    host_creds=cred,
    endpoint="/experimentids/{}/runs/{}/events".format(experiment_id, run_id),
    method="POST",
    json=end_event,
)

if response.status_code == 200:
    print(f"Successfully canceled a run with run id '{run_id}'")
else:
    print(f"Code: {response.status_code}, text: {response.text}")

Successfully run a run with run id '03ba6490-0889-4e28-a95e-9622dbb39c71'


## Write error message

In [27]:

# run = mlflow.start_run()
# print(run.info.run_id)

current_run = mlflow.active_run()
run_id = current_run.info.run_id
experiment_id = current_run.info.experiment_id

# get auth from client
client = MlflowClient()
cred = client._tracking_client.store.get_host_creds()
cred.host = cred.host.replace("mlflow/v2.0", "mlflow/v1.0").replace("mlflow/v1.0", "history/v1.0")

error_event = {
    "timestamp": datetime.utcnow().isoformat(),
    "name": "Microsoft.MachineLearning.Run.Error",
    "data": {
        "errorResponse":{
            "error": {
                "code": "UserError",
                "message": "test hod error",
            },
            "componentName": "promptflow"
        },
    },
}
response = http_request(
    host_creds=cred,
    endpoint="/experimentids/{}/runs/{}/events".format(experiment_id, run_id),
    method="POST",
    json=error_event,
)

if response.status_code == 200:
    print(f"Successfully write error message with run id '{run_id}'")
else:
    print(f"Code: {response.status_code}, text: {response.text}")

Successfully write error message with run id 'a9cc702b-fdb3-4f25-945c-c48a7800299f'


## Write run history

In [30]:
run = mlflow.start_run()
print(run.info.run_id)

current_run = mlflow.active_run()
run_id = current_run.info.run_id
experiment_id = current_run.info.experiment_id

# get auth from client
client = MlflowClient()
cred = client._tracking_client.store.get_host_creds()
cred.host = cred.host.replace("mlflow/v2.0", "mlflow/v1.0").replace("mlflow/v1.0", "history/v1.0")

properties = {
    "hod-test-property": "123"
}

response = http_request(
    host_creds=cred,
    endpoint="/experimentids/{}/runs/{}".format(experiment_id, run_id),
    method="PATCH",
    json={"runId": run_id, "properties": properties},
)

if response.status_code == 200:
    print(f"Successfully write run properties with run id '{run_id}'")
else:
    print(f"Code: {response.status_code}, text: {response.text}")

0996a1ef-294c-4c13-bb69-0a37c3516a92
Successfully write run properties with run id '0996a1ef-294c-4c13-bb69-0a37c3516a92'


In [32]:
mlflow.end_run()